In [ ]:
import pandas as pd
import numpy as np
import re

train = pd.read_csv('ML-1M_train_original.csv')

interaction_counts = train.groupby('item_id:token')['timestamp:float'].count().reset_index()
train = pd.merge(train, interaction_counts, on='item_id:token', how='left')
train.rename(columns={'timestamp:float_x': 'timestamp:float'}, inplace=True)
train.rename(columns={'timestamp:float_y': 'interaction_count'}, inplace=True)

train['title_genre'] = '<' + train['movie_title:token_seq'] + ' (genre: ' + train['genre:token_seq'] + ')>'
train['<movie_title:token_seq>'] = '<' + train['movie_title:token_seq'] + '>'

train = train.sort_values(by=['user_id:token', 'interaction_count'], ascending=[True, False])

np.set_printoptions(linewidth=np.inf)
np.random.seed(2024)

user_ids = np.unique(train['user_id:token'].values)
user_dict = dict()
rating_count = dict()

for user_id in user_ids:

    df_user = train[train['user_id:token'] == user_id]

    pos_5 = df_user[df_user['rating:float'] == 5]
    pos_4 = df_user[df_user['rating:float'] == 4]
    neg_3 = df_user[df_user['rating:float'] == 3]
    neg_2 = df_user[df_user['rating:float'] == 2]
    neg_1 = df_user[df_user['rating:float'] == 1]

    values_pos_5 = [pos_5['title_genre'].values]
    values_pos_4 = [pos_4['title_genre'].values]
    values_neg_3 = [neg_3['title_genre'].values]
    values_neg_2 = [neg_2['title_genre'].values]
    values_neg_1 = [neg_1['title_genre'].values]

    mean_rating = np.mean(df_user['rating:float'].values)

    values = [values_pos_5, values_pos_4, values_neg_3, values_neg_2, values_neg_1, mean_rating]
    user_dict[user_id] = values


train.head()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "4"

llm_dir = '/home/chwchong/_WWW25/LLM/'
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", cache_dir=llm_dir)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", cache_dir=llm_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
prompt1 = """
This dataset is from the MovieLens-1M dataset.

User's Positively Rated Movie List:
The following list includes movie titles and genres that the user has rated positively:
{positive_movies_5}

---------------------------------------------

Based on the User's Positively Rated Movie List, analyze the user's preferences and patterns.
You will be provided with Candidate List, which includes movie titles and genres that the user has rated both positively and negatively.
Your task is to strictly select movies and provide only the movie titles from Candidate List that the user is most likely to have rated positively.

Candidate List:
{candidate_example}

Output (Answer):
{only_title_4}

---------------------------------------------

Based on the User's Positively Rated Movie List, analyze the user's preferences and patterns.
You will be provided with Candidate List, which includes movie titles and genres that the user has rated both positively and negatively.
Your task is to strictly select movies and provide only the movie titles from Candidate List that the user is most likely to have rated positively.

Candidate List:
{negative_movies_32}

Output (Answer):
"""


def ask_llama1(question, tokenizer, model, device, stop_token="---------------------------------------------", max_occurrences=2):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    input_ids = inputs['input_ids']
    start_index = question.find("Candidate List:")
    if start_index != -1:
        second_start_index = question.find("Candidate List:", start_index + 1)
    else:
        second_start_index = -1
    if second_start_index != -1:
        candidate_list_2_text = question[second_start_index:]
    else:
        print('"Second occurrence of "Candidate List:" cannot be found.')
    inputs_sub = tokenizer(candidate_list_2_text, return_tensors="pt").to(device)
    input_ids_sub = inputs_sub['input_ids']
    length = input_ids.shape[1] + input_ids_sub.shape[1]
    if length > 5000:
        print(f"Skipping user due to input length: {input_ids.shape[1]}", end=", ")
        return None
    else:
        print(f"input length: {input_ids.shape[1]}", end=", ")
    outputs = model.generate(
        input_ids=input_ids.to(device),
        attention_mask=inputs['attention_mask'].to(device),
        max_length=length,
        pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
        num_beams=1,
        do_sample=False,
        temperature=1,
        top_p=1
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    stop_token_count = 0
    output_lines = []
    for line in response.split('\n'):
        if stop_token in line:
            stop_token_count += 1
            if stop_token_count > max_occurrences:
                break
        output_lines.append(line)
    return '\n'.join(output_lines).strip()

In [ ]:
prompt2 = """
This dataset is from the MovieLens-1M dataset.
Based on the User's Positively Rated Movie List, analyze the user's preferences and patterns.
You will be provided with Candidate List, which includes movie titles and genres that the user has rated both positively and negatively.
Your task is to strictly select movies and provide only the movie titles from Candidate List that the user is most likely to have rated positively.

User's Positively Rated Movie List:
{positive_movies_5}

Candidate List:
{candidate_example}

Output (Answer):
{only_title_4}

---------------------------------------------

This dataset is from the MovieLens-1M dataset.
Based on the User's Positively Rated Movie List, analyze the user's preferences and patterns.
You will be provided with Candidate List, which includes movie titles and genres that the user has rated both positively and negatively.
Your task is to strictly select movies and provide only the movie titles from Candidate List that the user is most likely to have rated positively.

User's Positively Rated Movie List:
{positive_movies_5}

Candidate List:
{negative_movies_32}

Output (Answer):
"""


def ask_llama2(question, tokenizer, model, device, stop_token="---------------------------------------------", max_occurrences=1):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    input_ids = inputs['input_ids']
    start_index = question.find("Candidate List:")
    if start_index != -1:
        second_start_index = question.find("Candidate List:", start_index + 1)
    else:
        second_start_index = -1
    if second_start_index != -1:
        candidate_list_2_text = question[second_start_index:]
    else:
        print('"Second occurrence of "Candidate List:" cannot be found.')
    inputs_sub = tokenizer(candidate_list_2_text, return_tensors="pt").to(device)
    input_ids_sub = inputs_sub['input_ids']
    length = input_ids.shape[1] + input_ids_sub.shape[1]
    if length > 5000:
        print(f"Skipping user due to input length: {input_ids.shape[1]}", end=", ")
        return None
    else:
        print(f"input length: {input_ids.shape[1]}", end=", ")
    outputs = model.generate(
        input_ids=input_ids.to(device),
        attention_mask=inputs['attention_mask'].to(device),
        max_length=length,
        pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
        num_beams=1,
        do_sample=False,
        temperature=1,
        top_p=1
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    stop_token_count = 0
    output_lines = []
    for line in response.split('\n'):
        if stop_token in line:
            stop_token_count += 1
            if stop_token_count > max_occurrences:
                break
        output_lines.append(line)
    return '\n'.join(output_lines).strip()

In [ ]:
# skip: Pre-removed users who have hallucinations because of too many interactions
skip = np.array([10, 18, 19, 22, 23, 26, 33, 36, 42, 45, 48, 53, 58, 62, 73, 90, 92, 117, 118, 123, 131, 136, 139, 146, 148, 149, 151, 157, 163, 166, 169, 173, 175, 181, 187, 192, 193, 195, 198, 199, 202, 204, 216, 223, 224, 225, 235, 238, 242, 245, 261, 264, 268, 271, 272, 284, 293, 301, 302, 308, 310, 314, 319, 321, 326, 327, 329, 331, 333, 338, 343, 346, 349, 352, 355, 366, 368, 386, 390, 392, 402, 403, 409, 411, 412, 415, 424, 426, 429, 438, 442, 445, 453, 454, 461, 474, 475, 476, 482, 509, 516, 518, 520, 524, 528, 531, 533, 543, 549, 550, 558, 563, 566, 570, 587, 588, 601, 604, 608, 624, 629, 631, 637, 639, 646, 647, 651, 655, 660, 669, 673, 676, 678, 687, 692, 696, 698, 699, 702, 710, 712, 713, 714, 716, 720, 721, 727, 731, 735, 737, 744, 746, 749, 752, 753, 757, 765, 770, 774, 777, 780, 791, 796, 798, 800, 801, 802, 808, 817, 822, 824, 839, 846, 850, 854, 855, 869, 877, 881, 889, 890, 897, 899, 904, 911, 919, 922, 929, 934, 935, 937, 948, 949, 955, 957, 963, 970, 971, 973, 975, 977, 981, 984, 996, 999, 1001, 1004, 1010, 1015, 1017, 1019, 1029, 1034, 1050, 1051, 1053, 1057, 1058, 1059, 1068, 1069, 1077, 1086, 1087, 1088, 1096, 1100, 1101, 1112, 1113, 1117, 1119, 1120, 1121, 1124, 1125, 1128, 1130, 1137, 1140, 1141, 1142, 1146, 1147, 1150, 1151, 1155, 1164, 1170, 1172, 1173, 1179, 1181, 1182, 1184, 1193, 1194, 1202, 1203, 1207, 1211, 1216, 1219, 1220, 1224, 1228, 1230, 1242, 1243, 1244, 1246, 1260, 1263, 1264, 1265, 1266, 1272, 1273, 1274, 1279, 1284, 1285, 1291, 1294, 1297, 1298, 1301, 1303, 1306, 1311, 1314, 1317, 1322, 1325, 1329, 1333, 1335, 1340, 1354, 1356, 1358, 1362, 1367, 1369, 1377, 1380, 1383, 1387, 1389, 1390, 1392, 1395, 1399, 1404, 1418, 1420, 1422, 1425, 1426, 1434, 1447, 1448, 1449, 1451, 1457, 1465, 1466, 1470, 1473, 1483, 1484, 1489, 1491, 1496, 1501, 1502, 1503, 1505, 1509, 1516, 1524, 1525, 1530, 1542, 1545, 1546, 1552, 1556, 1579, 1584, 1586, 1587, 1591, 1592, 1597, 1598, 1599, 1600, 1605, 1607, 1609, 1611, 1613, 1616, 1624, 1626, 1628, 1631, 1632, 1635, 1639, 1641, 1645, 1647, 1658, 1666, 1667, 1671, 1675, 1676, 1680, 1685, 1687, 1693, 1694, 1696, 1697, 1698, 1699, 1701, 1712, 1714, 1717, 1726, 1727, 1733, 1737, 1741, 1746, 1748, 1749, 1752, 1753, 1755, 1757, 1758, 1759, 1764, 1767, 1776, 1778, 1780, 1790, 1793, 1794, 1800, 1802, 1803, 1812, 1820, 1835, 1836, 1837, 1842, 1851, 1854, 1860, 1861, 1865, 1873, 1875, 1879, 1880, 1882, 1883, 1884, 1889, 1890, 1891, 1897, 1899, 1902, 1904, 1912, 1916, 1920, 1922, 1925, 1926, 1930, 1937, 1939, 1941, 1943, 1952, 1958, 1962, 1968, 1969, 1974, 1980, 1983, 1984, 1988, 1998, 2010, 2011, 2012, 2015, 2016, 2018, 2020, 2023, 2024, 2026, 2030, 2031, 2038, 2041, 2042, 2047, 2050, 2054, 2056, 2059, 2060, 2063, 2064, 2071, 2073, 2077, 2088, 2092, 2098, 2100, 2102, 2105, 2106, 2109, 2116, 2124, 2125, 2127, 2129, 2131, 2148, 2153, 2156, 2164, 2168, 2172, 2173, 2180, 2181, 2185, 2186, 2188, 2195, 2203, 2205, 2208, 2211, 2215, 2223, 2225, 2231, 2232, 2233, 2237, 2238, 2242, 2244, 2247, 2251, 2258, 2263, 2264, 2265, 2271, 2279, 2288, 2303, 2304, 2323, 2334, 2340, 2353, 2376, 2378, 2380, 2383, 2389, 2395, 2402, 2411, 2419, 2436, 2445, 2446, 2451, 2453, 2454, 2455, 2456, 2457, 2462, 2472, 2478, 2482, 2484, 2485, 2489, 2496, 2504, 2506, 2507, 2526, 2529, 2537, 2541, 2544, 2565, 2566, 2567, 2576, 2581, 2583, 2590, 2592, 2599, 2611, 2621, 2624, 2627, 2634, 2638, 2641, 2643, 2645, 2664, 2665, 2668, 2670, 2679, 2700, 2737, 2743, 2748, 2752, 2753, 2761, 2764, 2771, 2774, 2776, 2777, 2781, 2786, 2793, 2796, 2807, 2810, 2818, 2820, 2825, 2826, 2837, 2840, 2841, 2847, 2848, 2854, 2857, 2860, 2868, 2872, 2878, 2880, 2887, 2888, 2895, 2896, 2899, 2900, 2907, 2909, 2913, 2916, 2918, 2926, 2928, 2929, 2931, 2934, 2941, 2946, 2962, 2967, 2985, 2986, 2994, 2995, 2996, 3001, 3007, 3012, 3013, 3017, 3018, 3022, 3026, 3029, 3031, 3032, 3044, 3051, 3054, 3055, 3056, 3065, 3067, 3072, 3075, 3080, 3081, 3082, 3087, 3101, 3108, 3110, 3112, 3118, 3121, 3125, 3128, 3129, 3138, 3140, 3143, 3158, 3163, 3178, 3180, 3182, 3189, 3191, 3196, 3201, 3205, 3208, 3216, 3217, 3224, 3227, 3243, 3259, 3261, 3265, 3271, 3272, 3274, 3280, 3283, 3285, 3292, 3299, 3300, 3301, 3308, 3309, 3311, 3312, 3313, 3314, 3320, 3327, 3328, 3336, 3346, 3360, 3363, 3365, 3367, 3371, 3378, 3380, 3389, 3390, 3391, 3393, 3394, 3399, 3401, 3402, 3410, 3411, 3413, 3414, 3416, 3422, 3423, 3425, 3430, 3441, 3453, 3454, 3462, 3464, 3469, 3471, 3475, 3476, 3483, 3487, 3490, 3491, 3499, 3503, 3507, 3509, 3511, 3512, 3513, 3518, 3519, 3520, 3521, 3526, 3529, 3539, 3546, 3547, 3550, 3557, 3561, 3562, 3569, 3576, 3578, 3579, 3585, 3589, 3590, 3591, 3592, 3600, 3603, 3607, 3609, 3610, 3612, 3615, 3618, 3620, 3621, 3624, 3626, 3641, 3648, 3650, 3651, 3652, 3654, 3658, 3669, 3675, 3679, 3681, 3683, 3685, 3688, 3689, 3690, 3691, 3693, 3695, 3705, 3709, 3713, 3715, 3716, 3717, 3718, 3724, 3726, 3727, 3729, 3735, 3746, 3751, 3754, 3756, 3760, 3762, 3768, 3769, 3771, 3773, 3778, 3780, 3789, 3792, 3806, 3807, 3808, 3816, 3821, 3823, 3824, 3825, 3829, 3833, 3834, 3836, 3841, 3842, 3847, 3850, 3853, 3859, 3868, 3878, 3884, 3885, 3887, 3892, 3900, 3903, 3910, 3916, 3929, 3931, 3934, 3940, 3942, 3945, 3946, 3953, 3957, 3963, 3965, 3967, 3970, 3971, 3976, 3981, 3985, 3991, 3992, 3993, 3997, 3998, 3999, 4001, 4005, 4006, 4007, 4009, 4013, 4014, 4016, 4021, 4022, 4024, 4026, 4028, 4033, 4041, 4042, 4048, 4049, 4051, 4053, 4054, 4055, 4058, 4060, 4062, 4064, 4072, 4077, 4078, 4079, 4083, 4085, 4086, 4088, 4089, 4096, 4107, 4112, 4115, 4116, 4122, 4124, 4126, 4139, 4140, 4150, 4156, 4161, 4167, 4169, 4183, 4186, 4193, 4213, 4215, 4217, 4220, 4224, 4227, 4238, 4250, 4253, 4258, 4260, 4261, 4265, 4268, 4269, 4271, 4277, 4279, 4281, 4285, 4287, 4291, 4298, 4303, 4305, 4310, 4312, 4318, 4322, 4328, 4335, 4337, 4344, 4345, 4352, 4354, 4356, 4360, 4362, 4364, 4373, 4375, 4379, 4381, 4384, 4386, 4387, 4388, 4390, 4404, 4406, 4408, 4411, 4418, 4422, 4425, 4430, 4436, 4446, 4447, 4448, 4451, 4457, 4458, 4470, 4472, 4473, 4478, 4480, 4482, 4484, 4488, 4489, 4490, 4497, 4502, 4505, 4506, 4508, 4510, 4517, 4520, 4521, 4522, 4523, 4524, 4531, 4535, 4543, 4560, 4562, 4568, 4578, 4579, 4585, 4591, 4593, 4607, 4609, 4611, 4621, 4626, 4635, 4637, 4647, 4653, 4655, 4658, 4663, 4666, 4673, 4675, 4680, 4682, 4699, 4704, 4708, 4715, 4725, 4726, 4728, 4732, 4736, 4742, 4747, 4748, 4750, 4754, 4765, 4771, 4784, 4785, 4786, 4788, 4789, 4790, 4791, 4793, 4796, 4800, 4802, 4808, 4810, 4816, 4819, 4821, 4823, 4829, 4831, 4834, 4837, 4852, 4865, 4867, 4869, 4879, 4884, 4887, 4897, 4904, 4906, 4917, 4918, 4928, 4932, 4933, 4940, 4942, 4946, 4948, 4950, 4951, 4956, 4957, 4958, 4961, 4964, 4972, 4979, 4981, 4995, 5000, 5002, 5005, 5011, 5015, 5021, 5026, 5035, 5039, 5042, 5046, 5047, 5049, 5054, 5065, 5070, 5074, 5077, 5083, 5084, 5087, 5090, 5091, 5096, 5098, 5100, 5107, 5108, 5111, 5112, 5113, 5124, 5131, 5140, 5142, 5153, 5156, 5158, 5161, 5163, 5172, 5178, 5182, 5184, 5185, 5198, 5205, 5220, 5222, 5223, 5239, 5249, 5250, 5256, 5260, 5265, 5268, 5269, 5271, 5273, 5277, 5281, 5282, 5283, 5289, 5300, 5302, 5305, 5306, 5312, 5317, 5319, 5321, 5323, 5329, 5333, 5345, 5347, 5350, 5359, 5364, 5365, 5367, 5369, 5371, 5376, 5387, 5389, 5394, 5397, 5412, 5418, 5426, 5428, 5433, 5443, 5448, 5450, 5452, 5453, 5458, 5463, 5466, 5468, 5472, 5475, 5482, 5492, 5493, 5501, 5504, 5511, 5512, 5517, 5519, 5522, 5523, 5526, 5530, 5535, 5536, 5539, 5543, 5545, 5550, 5553, 5555, 5556, 5558, 5561, 5567, 5568, 5569, 5570, 5575, 5576, 5580, 5585, 5599, 5600, 5604, 5605, 5614, 5615, 5621, 5623, 5627, 5634, 5636, 5643, 5650, 5654, 5657, 5663, 5664, 5667, 5675, 5677, 5682, 5684, 5686, 5689, 5698, 5699, 5705, 5716, 5717, 5720, 5722, 5723, 5728, 5733, 5738, 5744, 5747, 5749, 5754, 5755, 5759, 5761, 5762, 5763, 5765, 5767, 5771, 5779, 5780, 5786, 5787, 5788, 5792, 5794, 5795, 5809, 5811, 5812, 5824, 5826, 5831, 5837, 5840, 5841, 5843, 5845, 5847, 5848, 5852, 5854, 5872, 5874, 5878, 5880, 5881, 5886, 5888, 5890, 5915, 5916, 5917, 5922, 5948, 5950, 5954, 5955, 5956, 5957, 5961, 5964, 5972, 5974, 5978, 5980, 5981, 5990, 5996, 6000, 6001, 6002, 6007, 6010, 6016, 6025, 6035, 6036, 6040])
user_ids = np.setdiff1d(np.array(list(user_dict.keys())), skip)
len(user_ids), user_ids

In [ ]:
skipped_users = []

with open('llama_distinguish_answer.txt', 'w') as f_cut, open('llama_distinguish_full.txt', 'w') as f_full:
    for user_id in user_ids:
        mean_rating = user_dict[user_id][5]
        positive_movies_5 = user_dict[user_id][0][0]
        positive_movies_4 = user_dict[user_id][1][0]
        negative_movies_3 = user_dict[user_id][2][0]
        negative_movies_2 = user_dict[user_id][3][0]
        negative_movies_1 = user_dict[user_id][4][0]

        print('mean_rating %.2f' %mean_rating, end=" ")
        if mean_rating >= 3:
            candidate_example = np.array(list(positive_movies_4) + list(negative_movies_2) + list(negative_movies_1))
            candidate_real = negative_movies_3
        else:
            candidate_example = np.array(list(positive_movies_4) + list(negative_movies_1))
            candidate_real = np.array(list(negative_movies_3) + list(negative_movies_2))
        np.random.shuffle(candidate_example)
        np.random.shuffle(candidate_real)

        only_title_4 = candidate_example[np.isin(candidate_example, positive_movies_4)]
        if len(only_title_4) > 0:
            only_title_4 = np.vectorize(lambda item: re.sub(r'\s*\(genre:.*$', '>', item))(only_title_4).astype(object)
        

        if len(candidate_real) > 5:
            chunked_candidate_real = [candidate_real[i:i+5] for i in range(0, len(candidate_real), 5)]
        else:
            chunked_candidate_real = [candidate_real]

        for i, chunk in enumerate(chunked_candidate_real):
            prompt = prompt1.format(
                positive_movies_5=positive_movies_5,
                candidate_example=candidate_example,
                only_title_4=only_title_4,
                negative_movies_32=chunk)
            response = ask_llama1(prompt, tokenizer, model, device)

            if response is None:
                print('user ' + str(user_id) + '[' + str(i) + '] skip')
                skipped_users.append(user_id)
                continue
            if response[-1] != ']':
                print('user ' + str(user_id) + '[' + str(i) + '] skip (hallucination)')
                prompt = prompt2.format(
                positive_movies_5=positive_movies_5,
                candidate_example=candidate_example,
                only_title_4=only_title_4,
                negative_movies_32=chunk)
                response = ask_llama2(prompt, tokenizer, model, device)
                if response is None:
                    print('user ' + str(user_id) + '[' + str(i) + '] skip')
                    skipped_users.append(user_id)
                    continue
                if response[-1] != ']':
                    print('user ' + str(user_id) + '[' + str(i) + '] skip (hallucination)')
                    skipped_users.append(user_id)
                    continue

            print('user ' + str(user_id) + '[' + str(i) + '] complete!')
            user_number_full = f"LLaMA's full recommendation for user {user_id}:"
            f_full.write(user_number_full + '\n')
            f_full.write(response + '\n\n\n\n\n\n\n\n\n')
            user_number_cut = f"LLaMA's cut recommendation for user {user_id}:" 
            f_cut.write(user_number_cut + '\n')
            f_cut.write(response[len(prompt)-1:] + '\n\n\n\n\n\n')


if skipped_users:
    skipped_users = np.unique(np.array(skipped_users))
    print(f"Skipped users due to input length: {skipped_users}")
    print(f"len(skipped_users): {len(skipped_users)}")
#2142m 53.8s